#  Explanation interactive notebook

Here is where all explanations for everything in each file will be

## src/en_str/mod.rs
In this file I set out the structs and enums I will be using in main.rs

***

### What is a Struct?
A struct is a collection of data that can be retrieved and changed. Structs can also implement functions however in this program they do not. For example:  
You could have a struct for a dog with the properties of name, age, and owner. In rust that would look like this:

In [ ]:
pub struct Dog {
    pub name: &str,
    pub age: u8,
    pub owner: String
}

***

### What is an Enum?
The idea behind an enum is that instead of using a basic data type like an integer to represent something, you should have specific states.

#### For example
You could represent colors with integers, Roy G Biv being 0, 1, 2, 3, 4, 5, 6, 7 but the problem is that there are more integers than just seven. It would be much better to make an Enum with each color being a different possible state. 

In [ ]:
pub enum Colors {
    Red,
    Orange,
    Yellow,
    Green,
    Blue,
    Indigo,
    Violet,
}

***

### Traits
Traits are an important part of creating your own data types.  
In rust, many traits can be generated for you automatically using the derive macro which looks like

In [ ]:
#[derive(Sample, Traits)]
pub enum /* or struct */ some_enum {
    some_value,
}

These autogenerated traits can be used to make functions on types automatically,  
for example:

In [ ]:
#[derive(Clone, Copy)]
pub struct some_struct {
    some_value: some_type,
}

This allows you to copy the value of some_struct without calling some_struct.clone()

***

### What enums and structs are made here
In this file there are three enums and one struct created.

##### The enums are as follows:
- Gender,  
- Type (as in type of stall needed), &  
- BathroomVariant (as in how you need to use the bathroom)  

#### The Struct is
- Person
##### To see the possible states of the enums and the properties of the struct click [here](src/en_str/mod.rs)



***

## src/main.rs
This file is where the actual simulation takes place

In this file there are three functions   
- **"main()"**, the required function for the program to run;  
- **"simulate()"**, the simulation;  
- **"check_satisfaction()"**, the way to check if every child has gone to the bathroom.

### main()
the main function does three things:  
1. It creates a list of a random number of people, 50 to 450 with random attributes  
2. It calls the simulate function on that list of people  
3. It keeps count of how many times the simulation has failed and succeeded and prints it to the console.
***

In [ ]:
loop {
    let mut people: Vec<Person> = vec![];
    // amount of people going to the bathroom
    let people_amt = rand::random::<u32>() % 400 + 51;

    for u in 1..people_amt {
        let gender: Gender;
        let variant: BathroomVariant;

        let random_variant: u32 = rand::random();
        let random_gender: u32 = rand::random();

        // 2 / 10 people have to feciate, the others urinate
        if random_variant % 10 >= 9 {
            variant = BathroomVariant::Feciate;
        } else {
            variant = BathroomVariant::Urinate;
        }
        // half of the people are males, the other half females
        if random_gender % 10 >= 5 {
            gender = Gender::Male;
        } else {
            gender = Gender::Female;
        }

        let time_to_get = rand::random::<u32>()%151+2;

        let time_remaining: u32 = match variant {
            BathroomVariant::Feciate => (rand::random::<u32>()%101+80)+time_to_get,
            BathroomVariant::Urinate => (rand::random::<u32>()%31+15)+time_to_get,
        };

        let stall_type: Type;

        // which type of stall should be used (only did this for ease of use later on)
        if gender == Gender::Female {
            stall_type = Type::FemaleStall;
        } else if variant == BathroomVariant::Feciate {
            stall_type = Type::Stall;
        } else {
            stall_type = Type::Urinal;
        }
        
        let vapor = rand::random::<u32>()%101;
        let is_vaping: bool;
        if vapor>75 {
            is_vaping = true;
        } else {
            is_vaping = false;
        }

        // creating a new person
        let person = Person {
            is_vaping,
            gender,
            variant,
            time_remaining,
            at_stall: false,
            finished: false,
            stall_type,
            index: u as usize -1 as usize,
        };
        // adding that person to the list of people
        people.push(person);
    }
    /*...*/
}

In this section we create a list of people and make a random amount of people with random attributes.   

***You may have noticed that I called .clone() on people, that is because it is a Vector of Person types and Vec<Person> does not implement the trait Copy.*** 
***

In [ ]:
fn main() {
    let mut failures: u32 = 0;
    let mut success: u32 = 0;
    loop {
        /* rest of the main function */
        // simulating those people and checking if it was a success
        let percent = simulate(people);
        if percent.clone() > 85.0 {
            success += 1;
        } else {
            failures += 1;
        }
        // printing to the console the amount of failures and successes
        println!("\nFailures: {:?}\nSuccesses: {:?}\nPercentage: {:?}", failures, success, percent);
    }
}

Here is where we call the simulate function, keep track of whether or not it was a success and print the result to the console.
***

### simulate()
the simulate function does too many things to go over each individual part but the main functionality goes like this:  
- for every person there is check if they can go into the bathrooms, if they are already in a bathroom, or if they are already finished
- if they're finished skip them
- if they're in a bathroom tick down their time remaining in the bathroom by one
- if they're vaping set their time to 180 once and make them use a stall
- if they're able to go to a bathroom put them into one
- if none of these are true they have to wait for a bathroom to open up
if you want to see the full function with comments go [here](src/main.rs#simulate-function)
***

### check_satisfaction()
the check_satisfaction function does one thing,  
- Check the percentage of students who were satisfied by the simulation.

In [ ]:
fn check_satisfaction(people: Vec<Person>) -> f32 {
    let mut finished_people: f32 = 0.0;
    let mut unfinished_people: f32 = 0.0;
    for person in people {
        match person.finished {
            true => finished_people += 1.0,
            false => unfinished_people += 1.0
        }
    }
    let percentage: f32 = finished_people/(unfinished_people+finished_people);
    percentage*100.0
}